In [67]:
import pandas as pd
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch import nn
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split


In [68]:
# Leer el archivo CSV
file_path = 'CorpusClasificado.csv'
df = pd.read_csv(file_path)

# Mapear clases a etiquetas numéricas
label_mapping = {"Académico": 0, "Administrativo": 1, "Conducta": 2, "Recursos": 3}
df['label'] = df['Categoria'].map(label_mapping)
# Reemplazar NaN por 0 en la columna 'label'
df['label'] = df['label'].fillna(0)

# Validar etiquetas únicas nuevamente
print("Etiquetas únicas después del reemplazo:", df['label'].unique())

# Validar etiquetas y eliminar datos inválidos
print("Etiquetas únicas:", df['label'].unique())
df = df[df['label'].isin([0, 1, 2, 3])]

# Dividir en conjunto de entrenamiento y prueba
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Segmento'], df['label'], test_size=0.2, random_state=42
)


Etiquetas únicas después del reemplazo: [0. 1. 2. 3.]
Etiquetas únicas: [0. 1. 2. 3.]


In [69]:
# Inicializar el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizar los textos
def tokenize_data(texts):
    return tokenizer(list(texts), padding='max_length', truncation=True, max_length=512, return_tensors="pt")

train_encodings = tokenize_data(train_texts)
test_encodings = tokenize_data(test_texts)

# Convertir etiquetas a tensores
train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())


In [71]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['label'] = self.labels[idx]
        return item

# Crear datasets
train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# Crear DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [72]:
class BERTGRUClassifier(nn.Module):
    def __init__(self, bert_model_name, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(BERTGRUClassifier, self).__init__()
        
        # Cargar modelo BERT preentrenado
        self.bert = BertModel.from_pretrained(bert_model_name)
        
        # Congelar capas inferiores de BERT (opcional, puedes ajustar según los datos disponibles)
        for param in self.bert.parameters():
            param.requires_grad = False  # Congelar todas las capas de BERT
            
        # Añadir GRU después de las salidas de BERT
        self.gru = nn.GRU(self.bert.config.hidden_size, hidden_dim, num_layers=n_layers, 
                          bidirectional=bidirectional, batch_first=True, dropout=dropout)
        
        # Dimensiones de salida de GRU
        gru_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.fc = nn.Linear(gru_output_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask, labels=None):
        # Salidas de BERT
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = bert_outputs.last_hidden_state  # [batch_size, seq_len, hidden_dim]
        
        # Pasar a través de GRU
        gru_output, _ = self.gru(last_hidden_state)
        
        # Usar la última salida de GRU
        pooled_output = gru_output[:, -1, :]
        logits = self.fc(self.dropout(pooled_output))
        
        # Calcular pérdida si se proporcionan etiquetas
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            return loss, logits
        return logits


In [73]:
# Instanciar el modelo
HIDDEN_DIM = 512
OUTPUT_DIM = 4
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = BERTGRUClassifier(
    bert_model_name='bert-base-uncased',
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM,
    n_layers=N_LAYERS,
    bidirectional=BIDIRECTIONAL,
    dropout=DROPOUT
)

# Mover a dispositivo GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Configurar el optimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Función de entrenamiento
def train_model(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device).long()
        optimizer.zero_grad()
        loss, _ = model(input_ids, attention_mask, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Función de evaluación
def eval_model(model, dataloader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device).long()
            loss, logits = model(input_ids, attention_mask, labels)
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return total_loss / len(dataloader), accuracy


In [74]:
# Entrenar el modelo
num_epochs = 40
for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, optimizer, device)
    eval_loss, eval_accuracy = eval_model(model, test_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Validation Loss: {eval_loss:.4f}")
    print(f"Validation Accuracy: {eval_accuracy:.4f}")


Epoch 1/40
Train Loss: 1.3497
Validation Loss: 1.4124
Validation Accuracy: 0.2222
Epoch 2/40
Train Loss: 1.2822
Validation Loss: 1.4833
Validation Accuracy: 0.2778
Epoch 3/40
Train Loss: 1.1796
Validation Loss: 1.5391
Validation Accuracy: 0.3333
Epoch 4/40
Train Loss: 1.2264
Validation Loss: 1.6036
Validation Accuracy: 0.3889
Epoch 5/40
Train Loss: 1.1511
Validation Loss: 1.6434
Validation Accuracy: 0.4444
Epoch 6/40
Train Loss: 1.2138
Validation Loss: 1.6688
Validation Accuracy: 0.4444
Epoch 7/40
Train Loss: 1.1828
Validation Loss: 1.6678
Validation Accuracy: 0.4444
Epoch 8/40
Train Loss: 1.1519
Validation Loss: 1.6614
Validation Accuracy: 0.3889
Epoch 9/40
Train Loss: 1.1412
Validation Loss: 1.6534
Validation Accuracy: 0.2778
Epoch 10/40
Train Loss: 1.1525
Validation Loss: 1.6354
Validation Accuracy: 0.3889
Epoch 11/40
Train Loss: 1.1038
Validation Loss: 1.6111
Validation Accuracy: 0.4444
Epoch 12/40
Train Loss: 1.1162
Validation Loss: 1.5968
Validation Accuracy: 0.3889
Epoch 13/40
T

In [75]:
# Guardar el modelo entrenado
torch.save(model.state_dict(), "Clasificador de Textos.pth")
print("Modelo guardado como 'Clasificador de Textos.pth'")


Modelo guardado como 'Clasificador de Textos.pth'


In [78]:
# Mapeo de clases
label_mapping = {0: "Académico", 1: "Administrativo", 2: "Conducta", 3: "Recursos"}

def predict_text_class(text):
    # Tokenizar el texto
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Hacer la predicción
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    return label_mapping[predicted_class], probs[0].tolist()

# Probar con un texto
texto_prueba = "FACULTAD DE CIENCIAS Y TECNOLOGÍA ING MIGUEL ORTIZ LIMÓN DECANO FACULTAD DE CIENCIAS Y TECNOLOGÍA ING VIDAL LOPEZ GONZALES DIRECTOR CARRERA INGENIERÍA QUÍMICA ING FELIX ZELAYA ACUÑA DIRECTOR CARRERA INGENIERÍA INDUSTRIAL DRA MA ELENA PALMA PHD DIRECTORA CARRERA INGENIERÍA AMBIENTAL ING FRANZ VILLALPANDO AMONZABEL DIRECTOR CARRERA INGENIERÍA DE SISTEMA MSC ING VÍCTOR HUGO GUTIÉRREZ VEGA DIRECTOR CARRERA INGENIERÍA DE PETRÓLEO Y GAS NATURAL ING JHONNY AVILES MIRANDA DIRECTOR CARRERA QUÍMICA INDUSTRIAL TS ING GONZALO BENITO PÉREZ SERRUDO DIRECTOR CARRERA INGENIERÍA DE ALIMENTOS ACTIVIDADES REALIZADAS Académicas Conclusión del proceso de autoevaluación de la Carrera de Ingeniería Ambiental siguiendo los procedimientos y metodologías del Comité Ejecutivo de la Universidad Boliviana CEUB Actualmente se está ejecutando el plan de mejoramiento de la Carrera para subsanar las observaciones y a futuro encarar el proceso de evaluación externa hacia su acreditación Todas las carreras de la Facultad de Ciencias y Tecnología están trabajando en el proceso de autoevaluación se espera concluir este proceso en la gestión 2021 INTERNACIONALIZACIÓN Bajo el convenio entre la Universidad de San Francisco Xavier y la Universidad de la Guajira de Colombia se viene ejecutando actividades como el I Coloquio Virtual Internacional de Ingeniería de Sistemas Proyecto de Aulas COVIIS 2020 que fomenta los trabajos de investigación y publicación de artículos entre estudiantes investigadores de ambos países conformando el comité científico internacional por docentes de ambas universidades en la calificación de trabajos Estas sinergias conjuntas proyectan un intercambio de conocimiento y en el marco del convenio se tiene previsto el intercambio de estudiantes y docentes inicialmente en forma virtual debido a efectos de la pandemia del Covid 19 sin embargo cuando mejoren las condiciones de salud estos intercambios podrían ser presenciales con el reconocimiento de los créditos cursados de acuerdo a la homologación del plan de estudios asimismo se trabaja en las cátedras internacionales y las clases espejo A través del Comité Ejecutivo del GEDC LATAM GLOBAL ENGINEERING DEANS COUNCIL Consejo Global de Decanos de Latinoamérica teniendo a Bolivia a la Universidad Mayor Real y Pontificia de San Francisco Xavier de Chuquisaca y a la Facultad de Ciencias y Tecnología como miembro de este comité y en asocio con la gran alianza del Consejo Federal de Decanos de Ingeniería Argentina CONFEDI la Asociación Colombiana de Facultades de Ingeniería ACOFI IFEESInternational Federation of Engineering Education Societies Federación Internacional de Sociedades de Educación en Ingeniería y LACCEILatin American and Caribbean Consortium of Engineering Institutions Consorcio Latinoamericano y del Caribe de Instituciones de Ingeniería y con la participación de varios expositores internacionales de reconocida trayectoria con temáticas actuales se abordaron las nuevas demandas para la Ingeniería latinoamericana los desafíos de la Universidad Latinoamericana la Ingeniería y los trabajos del futuro en estos procesos de adaptación de las universidades a los tiempos del Covid 19 y los efectos de la pandemia para dar respuestas a estas nuevas problemáticas EQUIPAMIENTO E INFRAESTRUCTURA Adquisición de equipamiento para el Laboratorio de Física Moderna con el financiamiento del Proyecto Sucre Ciudad Universitaria actualmente está en la etapa de pruebas para el beneficio del grado y del posgrado de la Especialidad Superior de Física versión I Entrega de equipamiento para las carreras de Ingeniería en Telecomunicaciones de un Data center y otros componentes importantes por las gestiones del Director de Carrera y la implementación en un nuevo Laboratorio para la carrera El proyecto de infraestructura en fase de pre inversión para la Facultad de Ciencias y Tecnología en predios de ex Refisur que se tenía previsto licitar este año 2020 y por efectos de pandemia del Covid 19 ha sido diferido para la gestión 2021 se espera su consolidación y proyección para un crecimiento sostenible de la población estudiantil en estos nuevos predios PROGRAMAS DE POSGRADO Desde la Unidad de Posgrado y Formación Continua se han desarrollado una serie de cursos cortos de formación continua a docentes y estudiantes de la Facultad en diferentes temáticas acordes a la coyuntura que se vive por efecto de la pandemia Covid 19 así mismo se ha lanzado "
clase, probabilidades = predict_text_class(texto_prueba)
print(f"Texto: {texto_prueba}")
print(f"Clase predicha: {clase}")
print(f"Probabilidades: {probabilidades}")


Texto: FACULTAD DE CIENCIAS Y TECNOLOGÍA ING MIGUEL ORTIZ LIMÓN DECANO FACULTAD DE CIENCIAS Y TECNOLOGÍA ING VIDAL LOPEZ GONZALES DIRECTOR CARRERA INGENIERÍA QUÍMICA ING FELIX ZELAYA ACUÑA DIRECTOR CARRERA INGENIERÍA INDUSTRIAL DRA MA ELENA PALMA PHD DIRECTORA CARRERA INGENIERÍA AMBIENTAL ING FRANZ VILLALPANDO AMONZABEL DIRECTOR CARRERA INGENIERÍA DE SISTEMA MSC ING VÍCTOR HUGO GUTIÉRREZ VEGA DIRECTOR CARRERA INGENIERÍA DE PETRÓLEO Y GAS NATURAL ING JHONNY AVILES MIRANDA DIRECTOR CARRERA QUÍMICA INDUSTRIAL TS ING GONZALO BENITO PÉREZ SERRUDO DIRECTOR CARRERA INGENIERÍA DE ALIMENTOS ACTIVIDADES REALIZADAS Académicas Conclusión del proceso de autoevaluación de la Carrera de Ingeniería Ambiental siguiendo los procedimientos y metodologías del Comité Ejecutivo de la Universidad Boliviana CEUB Actualmente se está ejecutando el plan de mejoramiento de la Carrera para subsanar las observaciones y a futuro encarar el proceso de evaluación externa hacia su acreditación Todas las carreras de la 